In [22]:
import os
import glob
import time

import GPRutils
import DESutils
import vK2KGPR
import vonkarmanFT as vk

import scipy.linalg
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import scipy.optimize as opt

import warnings
warnings.filterwarnings("ignore")

from importlib import reload

exps = DESutils.findExpNums()
gband = DESutils.bandDict["g"]
rband = DESutils.bandDict["r"]
iband = DESutils.bandDict["i"]
zband = DESutils.bandDict["z"]

In [29]:
expNum = zband[0]
expFile = "/home/fortino/GPRSolutions"
useRMS = True
curl = False
downselect = 1
minDESErr = -np.inf
maxDESErr = np.inf
vSet = "Subset A"
dualOpt = True
altOpt = False

In [30]:
reload(GPRutils)
t0 = time.time()*u.s
dataC = GPRutils.dataContainer()
dataC.load(expNum, downselect=downselect, minDESErr=minDESErr*u.mas**2, maxDESErr=maxDESErr*u.mas**2, useRMS=useRMS, vSet=vSet)
t1 = time.time()*u.s

Exposure: 348819
Band: z
Number of objects: 116592
INFO: Query finished. [astroquery.utils.tap.core]
There were 10207 matches within 0.5 arcsec.


In [31]:
reload(vK2KGPR)
GP = vK2KGPR.vonKarman2KernelGPR(dataC, printing=True, outDir=expFile, curl=curl)
# GP.fitCorr()
dataC.fitCorrParams = np.array([
    109.1565559,
    1.5349095,
    -0.0061141,
    0.0890528,
    -0.0355237  
])
t2 = time.time()*u.s

In [32]:
params = dataC.fitCorrParams

In [38]:
%%timeit
# Initialize a von Karman TurbulentLayer object. This is the object
# that you can use to input a set of parameters and get out a
# covariance function (the kernel function).
GP.ttt = vk.TurbulentLayer(
    variance=params[0],
    outerScale=params[1],
    diameter=params[2],
    wind=(params[3], params[4]))

10.1 s ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
# Calculate the covariance function of the von Karman model on this
# grid of points. This grid will represent all pairs of points within
# the training set.
du, dv = GPRutils.getGrid(GP.dC.Xtrain, GP.dC.Xtrain)
C = GP.ttt.getCuv(du, dv)

47.9 s ± 177 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit
# Reshape the covariance matrix from (N, N, 2, 2) to (2N, 2N).
n1, n2 = C.shape[0], C.shape[1]
K = np.swapaxes(C, 1, 2).reshape(2*n1, 2*n2)

1.84 s ± 4.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
# Calculate the White kernels.
W_GAIA, W_DES = GPRutils.makeW(
    GP.dC.Etrain_GAIA, GP.dC.Etrain_DES,
    useRMS=GP.dC.useRMS, curl=True)

4.34 s ± 55.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%timeit
L = np.linalg.cholesky(K + W_GAIA + W_DES)

alpha = np.linalg.solve(L, GPRutils.flat(GP.dC.Ytrain))
alpha = np.linalg.solve(L.T, alpha)

1min 20s ± 1.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
# Try using cho_factor and cho_solve.
cho_factor = scipy.linalg.cho_factor(K + W_GAIA + W_DES)
alphanew = scipy.linalg.cho_solve(cho_factor, GPRutils.flat(GP.dC.Ytrain))

26 s ± 449 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
np.all(np.isclose(alpha, alphanew))

True

In [ ]:
dataC.JackKnife(GP, dataC.fitCorrParams, fC=True)
t3 = time.time()*u.s

In [ ]:
# Wrapper function for keeping K and OS the same while leaving d, Wx, and Wy available as inputs.
# K and OS will be the fitCorr params.
FoM = lambda p: GP.figureOfMerit(np.array([*dataC.fitCorrParams[:2], *p]))

# Initial guess will be d, Wx, and Wy from fitCorr.
v0 = dataC.fitCorrParams[2:]

# Call the GP optimize method with the different figure of merit function
GP.optimize(v0=v0, func=FoM, ftol=0.05, maxfun=50)
d_Wx_Wy = GP.opt_result_GP[0].copy()

In [ ]:
# Wrapper function for keeping d, Wx, and Wy the same while leaving K and OS available as inputs.
# d, Wx, and Wy will be the values from the first stage of optimization.
FoM = lambda p: GP.figureOfMerit(np.array([*p, *d_Wx_Wy]))

# Initial guess will be K and OS from fitCorr.
v0 = dataC.fitCorrParams[:2]

# Call the GP optimize method with the different figure of merit function
GP.optimize(v0=v0, func=FoM, ftol=0.05, maxfun=50)
K_oS = GP.opt_result_GP[0].copy()

dataC.params = np.array([*K_oS, *d_Wx_Wy])

In [ ]:
dataC.JackKnife(GP, dataC.params)
t5 = time.time()*u.s

In [ ]:
dataC.saveFITS(expFile)
tf = time.time()*u.s

In [ ]:
total_time = tf - t0
load_time = t1 - t0
fC_time = t2 - t1
fC_jackknife_time = t3 - t2
opt_time = t5 - t4
opt_jackknife_time = t6 - t7

with open(GP.paramFile, mode="a+") as file:
    file.write(f"Total Time                          : {total_time.to(u.hr):.3f}")
    file.write(f"Load Time                           : {load_time.to(u.min):.3f}")
    file.write(f"Correlation Fitting Time            : {fC_time.to(u.min):.3f}")
    file.write(f"Correlating Fitting Jackknife Time  : {fC_jackknife_time.to(u.min):.3f}")
    file.write(f"Optimization Time                   : {opt_time.to(u.hr):.3f}")
    file.write(f"Optimization Jackknife Time         : {opt_jackknife_time.to(u.min):.3f}")